In [2]:
#Keep!!!!!!

# pip install pyscf
from pyscf import gto, scf, mcscf, cc
import numpy as np

# --- 1) 대략적 Gas-phase LiCoO2 모형 좌표 (데모용) ---
# Co–O ~1.93 Å로 O를 축 방향에 두고, Li는 축 바깥쪽에 배치
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

def CASCI(C,L,theta):
    Co = (0.0, 0.0, 0.0)
    O1 = (C, 0.0, 0.0)
    O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
    Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

    geom = f"""
    Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
    O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
    O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
    Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
    """

    # --- 2) 분자/기저/전자상태 설정 ---
    # LiCoO2 벌크에선 Co3+ (d6, low-spin)이라 싱글렛이 자연스러움 → spin=0
    mol = gto.Mole()
    mol.atom  = geom
    mol.basis = 'sto-3g'      # 데모: 가벼운 전자구조 테스트용. 필요하면 def2-TZVP/def2-SVP+ECP 등으로 교체
    mol.charge = 0
    mol.spin   = 0              # 2S (singlet=0)
    mol.verbose = 4
    mol.unit = 'Ang'
    mol.build()

    # --- 3) SCF (수렴 보조 옵션 약간) ---
    mf = scf.RHF(mol)
    mf.max_cycle = 200
    mf.level_shift = 0.5      # 초기 수렴 보조 (필요 시 0.0으로)
    mf.damp = 0.2
    e_hf = mf.kernel()
    if not mf.converged:
        raise RuntimeError("SCF not converged")

    # --- 4) CASCI 설정 ---
    # Co3+ d6 (저스핀) 근방만 간단히 다루고 싶다면: ncas=6, nelecas=6 (대략 t2g-유사 전자수 느낌)
    # 더 넓게 Co 3d(+약간의 O 2p)까지 보려면 ncas=10~12 정도로 확장.
    ncas, nelecas = 12, 16
    mc = mcscf.CASCI(mf, ncas, nelecas)
    mc.kernel()                 # <-- 언패킹 금지
    e_cas = mc.e_tot            # 총 에너지
    ci    = mc.ci               # FCI 벡터

    print("\n=== Results ===")
    print(f"RHF energy  = {e_hf:.10f} Eh")
    print(f"CASCI energy= {e_cas:.10f} Eh")

    # --- 5) 액티브 자연점유수(확인용) ---
    # (CASCI에서도 fcisolver로 1RDM 가능)
    rdm1 = mc.fcisolver.make_rdm1(ci, ncas, nelecas)
    occ, _ = np.linalg.eigh(rdm1)
    print("Active natural occupations:", np.round(np.sort(occ)[::-1], 6))
    return e_cas

def CCSD(C,L,theta):
    Co = (0.0, 0.0, 0.0)
    O1 = (C, 0.0, 0.0)
    O2 = (C*np.cos(theta), C*np.sin(theta), 0.0)
    Li = (C + L*np.cos(np.pi - theta), -L*np.sin(np.pi - theta), 0.0)

    geom = f"""
    Co   {Co[0]:.8f}   {Co[1]:.8f}   {Co[2]:.8f}
    O    {O1[0]:.8f}   {O1[1]:.8f}   {O1[2]:.8f}
    O    {O2[0]:.8f}   {O2[1]:.8f}   {O2[2]:.8f}
    Li   {Li[0]:.8f}   {Li[1]:.8f}   {Li[2]:.8f}
    """

    mol = gto.M(
        atom=geom, unit="Ang",
        basis="sto-3g",
        charge=0, spin=0,   # 싱글렛
        verbose=4
    )

    # --- 2) HF (RHF) ---
    mf = scf.RHF(mol).run()

    # --- 3) CCSD ---
    #  (필요하면 core 동결: mycc = cc.CCSD(mf, frozen=1)  # 가장 아래 MO 1개 동결)
    mycc = cc.CCSD(mf)
    ecc, t1, t2 = mycc.kernel()     # CCSD correlation energy (Eh)

    # (선택) perturbative triples (T)
    et = mycc.ccsd_t()              # (T) correction (Eh)

    # --- 4) 결과 요약 ---
    E_CCSD     = mf.e_tot + ecc
    E_CCSD_T   = E_CCSD + et

    print("\n=== CCSD Summary ===")
    print(f"HF energy        : {mf.e_tot:.10f} Eh")
    print(f"CCSD corr. (E_c) : {ecc:.10f} Eh")
    print(f"(T) correction   : {et:.10f} Eh")
    print(f"Total CCSD       : {E_CCSD:.10f} Eh")
    print(f"Total CCSD(T)    : {E_CCSD_T:.10f} Eh")
    
    return E_CCSD 

In [3]:
#CCSD
# pip install pyscf


# --- 1) 분자 좌표 (원하는 XYZ로 교체 가능) ---
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)



# T1 diagnostic (대략 0.02 이하이면 단일참조 양호한 편으로 흔히 봄)
#print(f"T1 diagnostic    : {mycc.t1_diag():.4f}")

In [4]:
import numpy as np

x = [1, 0.94, 0.78, 0.75]

C = [1.9220, 1.9210, 1.9132, 1.9059]
L = [2.0946, 2.0944, 2.0700, 2.0700]
theta = [
    np.deg2rad(94.24),
    np.deg2rad(94.27),
    np.deg2rad(94.72),
    np.deg2rad(95.07),
]
E_CASCI_list = []
E_CCSD_list = []

for i in range(4):
    c = C[i]
    l = L[i]
    the = theta[i]
    E_CASCI = CASCI(c,l,the)
    E_CCSD = CCSD(c,l,the)
    E_CASCI_list.append(E_CASCI)
    E_CCSD_list.append(E_CCSD)
    
print(E_CASCI_list)
print(E_CCSD_list)

System: uname_result(system='Darwin', node='choeyunhoui-Macmini.local', release='24.6.0', version='Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:34 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T8103', machine='arm64')  Threads 1
Python 3.11.13 (main, Jun  5 2025, 08:21:08) [Clang 14.0.6 ]
numpy 1.26.4  scipy 1.15.3  h5py 3.14.0
Date: Wed Sep  3 11:10:39 2025
PySCF version 2.9.0
PySCF path  /opt/miniconda3/envs/hivqe/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 46
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = Ang
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Co     0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 O      1.922000000000   0.000000000000   0.

In [5]:
# x = 1
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

# # x= 0.94
# C = 1.9210
# L = 2.0944
# theta = np.deg2rad(94.27)

# #x = 0.78
# C = 1.9132
# L = 2.07
# theta = np.deg2rad(94.72)

# #x = 0.75
# C = 1.9059
# L = 2.07
# theta = np.deg2rad(95.07)


C = []